# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [1]:
import pandas as pd
import requests
from geopy import distance

In [2]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php

# TODO join station data to tube line
london_stations_df = pd.read_csv('london_stations.csv')

In [3]:
LONDON_STATIONS = []
for x in london_stations_df[['Station', 'Latitude', 'Longitude']].values:
    LONDON_STATIONS.append((x[0],(x[1],x[2])))

In [4]:
LONDON_STATIONS[:5] #preview data

[('Abbey Road', (51.5319519867171, 0.00372337109717115)),
 ('Abbey Wood', (51.490784095703106, 0.120271969782359)),
 ('Acton Central', (51.5087577801896, -0.263430199363473)),
 ('Acton Main Line', (51.5168869310483, -0.267689952184077)),
 ('Acton Town', (51.503071442963495, -0.28030270140601))]

In [5]:
# test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [5]:
# TODO refactor - use less code
# TODO Add documentation
class NearestStation:
        
    def __init__(self, postcode, unit='m', stations=LONDON_STATIONS):
        self.postcode = postcode.lower()
        self.lat_lon = self.geo_dict['lat_lon']
        self._unit = unit
        self.stations = stations
        
    @property
    def unit(self):
        return self._unit
    
    @unit.setter
    def unit(self, new_unit):
        if new_unit in ['m', 'km']:
            self._unit = new_unit
        else:
            raise ValueError('Enter "m" for miles or "km" for kilometers')
        
    @property
    def endpoint(self):
        pc = self.postcode.replace(' ','')
        return f'http://api.postcodes.io/postcodes/{pc}'
    
    @property
    def geo_dict(self):
        geocoded_dict = {}
        r = requests.get(self.endpoint)
        geocoded_dict['postcode'] = r.json()['result']['postcode']
        geocoded_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
        return geocoded_dict
    
    @property
    def distances(self):
        if self._unit == 'm':
            distances = [(x[0], round(distance.distance(self.lat_lon, x[1]).miles,1)) for x in self.stations]
            distances.sort(key=lambda x:x[1])
            return distances
        elif self._unit == 'km':
            distances = [(x[0], round(distance.distance(self.lat_lon, x[1]).km,1)) for x in self.stations]
            distances.sort(key=lambda x:x[1])
            return distances
    
    def find_nearest_station(self): 
        return (self.distances[0])

    def find_nearest_n_stations(self,n):
        return (self.distances[:n])
    
    def find_stations_with_x_radius(self, n):
        return [x for x in self.distances if x[1] <= n]

In [10]:
x = NearestStation('N4 4AF', stations = LONDON_STATIONS[:5])

In [11]:
x.geo_dict

{'postcode': 'N4 4AF', 'lat_lon': (51.573188, -0.10576)}

In [12]:
x.lat_lon

(51.573188, -0.10576)

In [13]:
x.endpoint

'http://api.postcodes.io/postcodes/n44af'

In [14]:
x.postcode

'n4 4af'

In [15]:
print(x.find_nearest_station(), "\n", x.unit)

('Abbey Road', 5.5) 
 m


In [16]:
x.distances

[('Abbey Road', 5.5),
 ('Acton Main Line', 8.0),
 ('Acton Central', 8.1),
 ('Acton Town', 9.0),
 ('Abbey Wood', 11.3)]

In [14]:
x.unit

'm'

In [15]:
x.unit ='km' #should change unit

In [16]:
x.unit

'km'

In [17]:
x.distances

[('Harringay', 0.5),
 ('Harringay Green Lanes', 0.7),
 ('Manor House', 0.7),
 ('Crouch Hill', 0.8),
 ('Finsbury Park', 1.0),
 ('Hornsey', 1.5),
 ('Arsenal', 1.6),
 ('Turnpike Lane', 1.9),
 ('Upper Holloway', 1.9),
 ('Stamford Hill', 2.0),
 ('Archway', 2.2),
 ('Drayton Park', 2.2),
 ('Holloway Road', 2.3),
 ('South Tottenham', 2.5),
 ('Stoke Newington', 2.5),
 ('Seven Sisters', 2.6),
 ('Wood Green', 2.7),
 ('Caledonian Road', 2.9),
 ('Canonbury', 2.9),
 ('Highgate', 2.9),
 ('Tufnell Park', 2.9),
 ('Alexandra Palace', 3.0),
 ('Highbury and Islington', 3.0),
 ('Rectory Road', 3.0),
 ('Bruce Grove', 3.4),
 ('Caledonian Road and Barnsbury', 3.4),
 ('Dalston Kingsland', 3.4),
 ('Kentish Town', 3.5),
 ('Clapton', 3.6),
 ('Tottenham Hale', 3.6),
 ('Dalston Junction', 3.7),
 ('Essex Road', 3.7),
 ('Gospel Oak', 3.7),
 ('Bowes Park', 3.9),
 ('Bounds Green', 4.0),
 ('Hackney Downs', 4.1),
 ('Kentish Town West', 4.1),
 ('Camden Road', 4.2),
 ('White Hart Lane', 4.3),
 ('East Finchley', 4.4),
 ('Ha

In [18]:
print(x.find_nearest_station(), "\n", x.unit)

('Harringay', 0.5) 
 km


In [19]:
# find within 1 mile of station
x.find_stations_with_x_radius(2)
# [x for x in x.find_nearest_n_stations(10) if x[1]<=1]

[('Harringay', 0.5),
 ('Harringay Green Lanes', 0.7),
 ('Manor House', 0.7),
 ('Crouch Hill', 0.8),
 ('Finsbury Park', 1.0),
 ('Hornsey', 1.5),
 ('Arsenal', 1.6),
 ('Turnpike Lane', 1.9),
 ('Upper Holloway', 1.9),
 ('Stamford Hill', 2.0)]

In [20]:
NearestStation('e17 3ss').find_nearest_station()

('Wood Street', 0.4)

In [21]:
# Clean station data?
NearestStation('e17 3ss').find_nearest_n_stations(10)

[('Wood Street', 0.4),
 ('Highams Park', 1.1),
 ('Walthamstow Central', 1.1),
 ('Snaresbrook', 1.2),
 ('South Woodford', 1.2),
 ('Walthamstow Queens Road', 1.3),
 ('Leyton Midland Road', 1.6),
 ('St James Street', 1.6),
 ('Leytonstone', 1.7),
 ('Wanstead', 1.7)]

In [22]:
NearestStation('e17 3ss').find_stations_with_x_radius(1.5)

[('Wood Street', 0.4),
 ('Highams Park', 1.1),
 ('Walthamstow Central', 1.1),
 ('Snaresbrook', 1.2),
 ('South Woodford', 1.2),
 ('Walthamstow Queens Road', 1.3)]